# Spike and Burst Analysis with Visibility Graphs

This notebook demonstrates how to detect spikes and bursts in an ABF file, classify bursts into different types, and visualize the results. The workflow goes from raw electrophysiological data to spike detection, burst detection, classification, saving the results, and plotting them for interpretation.

In [ ]:
# Load libraries
import pyabf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
%matplotlib widget

## Step 1: Load ABF file and concatenate sweeps
ABF (Axon Binary File) is a format used in electrophysiology for patch-clamp or intracellular recordings.

Here we load the ABF file and concatenate all sweeps into a single continuous voltage trace. This way, the data can be analyzed as one long recording instead of multiple separate sweeps.

**Expected output:** A message printing the file name, the number of sweeps, and the total duration of the recording in seconds.

In [ ]:
file_path = "bursting/cell89basal.abf"
abf = pyabf.ABF(file_path)

signal = np.concatenate([abf.setSweep(i) or abf.sweepY for i in range(abf.sweepCount)])
dt = 1.0 / abf.dataRate
time = np.arange(len(signal)) * dt

print(f"File: {file_path} | sweeps: {abf.sweepCount} | total length: {time[-1]:.2f} s")

## Step 2: Spike detection
Spikes are detected using a voltage threshold. Any rapid voltage increase above `-35 mV` is considered a spike.

- `find_peaks` locates peaks in the signal above the threshold.
- We then extract the spike times in seconds.

**Expected output:** Two arrays:
- `spike_indices`: indices of spikes in the signal.
- `spike_times`: corresponding times in seconds.

In [ ]:
threshold = -35  # mV
spike_indices, _ = find_peaks(signal, height=threshold)
spike_times = time[spike_indices]

## Step 3: Burst detection
Bursts are defined as sequences of spikes where the time between spikes (interspike interval, ISI) is shorter than a given threshold (here, 0.3 seconds).

The algorithm:
1. Compute ISI = time differences between consecutive spikes.
2. Group spikes together if their ISI < threshold.
3. Save bursts as start and end times.

**Expected output:** A list of bursts, each represented by a `(start_time, end_time)` tuple.

In [ ]:
isi = np.diff(spike_times)
burst_threshold = 0.3  # seconds

bursts = []
current_burst = [spike_times[0]]
for i in range(1, len(isi)):
    if isi[i-1] < burst_threshold:
        current_burst.append(spike_times[i])
    else:
        if len(current_burst) > 1:
            bursts.append((current_burst[0], current_burst[-1]))
        current_burst = [spike_times[i]]
if len(current_burst) > 1:
    bursts.append((current_burst[0], current_burst[-1]))

## Step 4: Burst classification
Now that bursts are detected, we try to classify them based on their voltage profile:

- **Square Wave bursts**: if the minimum voltage inside the burst is higher than the surrounding baseline.
- **Parabolic bursts**: if the minimum voltage is lower than the baseline.
- **Other**: if no clear pattern is found.

This is a simplified heuristic that uses the average of the voltages before and after the burst as a baseline.

**Expected output:** Three lists: `square_wave_bursts`, `parabolic_bursts`, `other_bursts`.

In [ ]:
square_wave_bursts, parabolic_bursts, other_bursts = [], [], []

for i, (burst_start, burst_end) in enumerate(bursts):
    burst_mask = (time >= burst_start) & (time <= burst_end)
    burst_min = np.min(signal[burst_mask])

    prev_mean = np.mean(signal[(time > bursts[i-1][1]) & (time < burst_start)]) if i>0 else np.nan
    next_mean = np.mean(signal[(time > burst_end) & (time < bursts[i+1][0])]) if i < len(bursts)-1 else np.nan
    inter_mean = np.nanmean([prev_mean, next_mean])

    if burst_min > inter_mean:
        square_wave_bursts.append((burst_start, burst_end))
    elif burst_min < inter_mean:
        parabolic_bursts.append((burst_start, burst_end))
    else:
        other_bursts.append((burst_start, burst_end))

## Step 5: Save burst information to CSV
Each burst is saved with an ID, start time, end time, and classification type.

This makes it easy to use the burst data in further analysis or visualization pipelines.

**Expected output:** A CSV file (`burst_info_cell89.csv`) and a printed DataFrame summarizing all bursts and their classifications.

In [ ]:
burst_list = []

for idx, (start, end) in enumerate(square_wave_bursts):
    burst_list.append([idx+1, start, end, "Square Wave"])
for idx, (start, end) in enumerate(parabolic_bursts):
    burst_list.append([idx+1+len(square_wave_bursts), start, end, "Parabolic"])
for idx, (start, end) in enumerate(other_bursts):
    burst_list.append([idx+1+len(square_wave_bursts)+len(parabolic_bursts), start, end, "Other"])

df_bursts = pd.DataFrame(burst_list, columns=["Burst_Number", "Start_Time_s", "End_Time_s", "Type"])
df_bursts.to_csv("burst_info_cell89.csv", index=False)
print(df_bursts)

## Step 6: Visualization
Finally, we plot the results:
- The full voltage trace is plotted.
- Spikes are shown as red dots.
- Bursts are highlighted with colored shaded regions:
  - Blue for Square Wave
  - Green for Parabolic
  - Orange for Other

**Expected output:** A figure showing the raw trace with spike times and burst classifications overlaid, making it easy to visually inspect the detection and classification results.

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(time, signal, lw=0.2)
plt.plot(spike_times, signal[spike_indices], 'r.', markersize=3)

for start, end in square_wave_bursts:
    plt.axvspan(start, end, color='blue', alpha=0.3)
for start, end in parabolic_bursts:
    plt.axvspan(start, end, color='green', alpha=0.3)
for start, end in other_bursts:
    plt.axvspan(start, end, color='orange', alpha=0.3)

plt.xlabel("Time (s)")
plt.ylabel("Voltage (mV)")
plt.title("Detected Spikes and Burst Types")
plt.show()